# Collaborative filtering on the MovieLens Dataset

###### This notebook is based on part of Chapter 9 of [BigQuery: The Definitive Guide](https://www.oreilly.com/library/view/google-bigquery-the/9781492044451/ "http://shop.oreilly.com/product/0636920207399.do") by Lakshmanan and Tigani.
### MovieLens dataset
To illustrate recommender systems in action, let’s use the MovieLens dataset. This is a dataset of movie reviews released by GroupLens, a research lab in the Department of Computer Science and Engineering at the University of Minnesota, through funding by the US National Science Foundation.

Download the data and load it as a BigQuery table using:

In [4]:
import os
PROJECT = "replace-with-your-project-id" # REPLACE WITH YOUR PROJECT ID
BUCKET = "your-bucket-here" # REPLACE WITH YOUR BUCKET NAME
REGION = "us-central1" # REPLACE WITH YOUR BUCKET REGION e.g. us-central1

# Do not change these
os.environ["PROJECT"] = PROJECT
os.environ["BUCKET"] = BUCKET
os.environ["REGION"] = REGION
os.environ["TFVERSION"] = "2.0"

### Download 20 million movie ratings

The grouplens.org site hosts a zip file of 20 million movie `ratings` as well as a `movies` lookup table. Run the below to download the zip locally and to preview our raw data.

In [10]:
%%bash
# Create a local directory for the .zip of 20 Million records of movielens data
rm -r -f bqml_data
mkdir bqml_data
cd bqml_data
curl -O 'http://files.grouplens.org/datasets/movielens/ml-20m.zip'
unzip ml-20m.zip
ls

# show some of the raw data
printf "\n\n\033[32;1m Let's see some movie data... \033[0m\n\n";
head --lines=20 'ml-20m/movies.csv'

printf "\n\n\033[32;1m Let's see some ratings data... \033[0m\n\n";
head --lines=10 'ml-20m/ratings.csv'

Archive:  ml-20m.zip
   creating: ml-20m/
  inflating: ml-20m/genome-scores.csv  
  inflating: ml-20m/genome-tags.csv  
  inflating: ml-20m/links.csv        
  inflating: ml-20m/movies.csv       
  inflating: ml-20m/ratings.csv      
  inflating: ml-20m/README.txt       
  inflating: ml-20m/tags.csv         
ml-20m
ml-20m.zip


 Let's see some movie data... 

movieId,title,genres
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy
6,Heat (1995),Action|Crime|Thriller
7,Sabrina (1995),Comedy|Romance
8,Tom and Huck (1995),Adventure|Children
9,Sudden Death (1995),Action
10,GoldenEye (1995),Action|Adventure|Thriller
11,"American President, The (1995)",Comedy|Drama|Romance
12,Dracula: Dead and Loving It (1995),Comedy|Horror
13,Balto (1995),Adventure|Animation|Children
14,Nixon (1995),Drama
15,Cutthroat Isla

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  189M  100  189M    0     0  41.0M      0  0:00:04  0:00:04 --:--:-- 44.1M


### Load movie ratings and metadata into new `movielens` BigQuery dataset

Run the below bq command line script to:
1. create a `movielens` BigQuery dataset if it doesnt exist
2. load the `ratings` and `movies` into two new tables

__Note:__ This loading process will take 3 - 5 minutes

In [13]:
%%bash
# Create the `movielens` dataset if it doesnt exist already
datasetexists=$(bq ls -d | grep -w movielens)

if [ -n "$datasetexists" ]; then
    echo -e "BigQuery dataset movielens already exists, let's not recreate it."

else
    echo "Creating BigQuery dataset titled: movielens"
    
    bq --location=US mk --dataset \
        --description 'Movie Recommendations' \
        $PROJECT:movielens
   echo "\nHere are your current datasets:"
   bq ls
fi
    
# Load the movie `ratings` and `movies` CSVs into BigQuery
bq --location=US \
load \
    --replace=true \
    --source_format=CSV \
    --autodetect movielens.ratings bqml_data/ml-20m/ratings.csv

bq --location=US \
load \
    --replace=true \
    --source_format=CSV \
    --autodetect movielens.movies_raw bqml_data/ml-20m/movies.csv

BigQuery dataset movielens already exists, let's not recreate it.






Waiting on bqjob_r24722e65f65f835d_0000016de104142b_1 ... (2s) Current status: DONE    

## Exploring the data 
Two tables should now be available in <a href="https://console.cloud.google.com/bigquery">BigQuery</a>. Run the below query which uses BigQuery's stored metadata about your dataset to return it's size and number of rows.

In [16]:
%%bigquery --project $PROJECT
SELECT 
  dataset_id,
  table_id,
  -- Convert bytes to GB.
  ROUND(size_bytes/pow(10,9),2) as size_gb,
  -- Convert UNIX EPOCH to a timestamp.
  TIMESTAMP_MILLIS(creation_time) AS creation_time,
  TIMESTAMP_MILLIS(last_modified_time) as last_modified_time,
  row_count,
  CASE 
    WHEN type = 1 THEN 'table'
    WHEN type = 2 THEN 'view'
  ELSE NULL
  END AS type
FROM
  movielens.__TABLES__
ORDER BY size_gb DESC

,dataset_id,table_id,size_gb,creation_time,last_modified_time,row_count,type
0,movielens,ratings,0.64,2019-10-18 22:05:30.258000+00:00,2019-10-18 22:36:34.421000+00:00,20000263,table
1,movielens,movies_raw,0.00,2019-10-18 22:36:47.777000+00:00,2019-10-18 22:36:47.777000+00:00,27278,table


As expected with the ml-20m dataset, we have 20 million rows of ratings data on ~27k movies. Great!

### Understanding ratings and recommendations

[Collaborative filtering](https://en.wikipedia.org/wiki/Collaborative_filtering) provides a way to generate product recommendations for users, or user targeting for products. The starting point is a table, `movielens.ratings`, with three columns: a user id, an item id, and the rating that the user gave the product. This table can be sparse -- users don’t have to rate all products. Then, based on just the ratings, the technique finds similar users and similar products and determines the rating that a user would give an unseen product. Then, we can recommend the products with the highest predicted ratings to users, or target products at users with the highest predicted ratings.

Let's take a look at the ratings data:

In [21]:
%%bigquery --project $PROJECT
SELECT 
    *, 
    TIMESTAMP_SECONDS(timestamp) AS rated_on
FROM movielens.ratings
LIMIT 10

,userId,movieId,rating,timestamp,rated_on
0,8,185,2.0,834586861,1996-06-12 13:41:01+00:00
1,18,1958,2.0,1195573225,2007-11-20 15:40:25+00:00
2,46,333,2.0,846354776,1996-10-26 18:32:56+00:00
3,56,3326,2.0,977502624,2000-12-22 16:30:24+00:00
4,100,427,2.0,835720568,1996-06-25 16:36:08+00:00
5,104,2590,2.0,944929472,1999-12-11 16:24:32+00:00
6,104,3153,2.0,944919977,1999-12-11 13:46:17+00:00
7,108,172,2.0,836569780,1996-07-05 12:29:40+00:00
8,128,153,2.0,900721433,1998-07-18 00:23:53+00:00
9,129,80748,2.0,1287115184,2010-10-15 03:59:44+00:00


A quick exploratory query yields that the dataset consists of over 138 thousand users, nearly 27 thousand movies, and a little more than 20 million ratings, confirming that the data has been loaded successfully.

In [15]:
%%bigquery --project $PROJECT
SELECT 
    COUNT(DISTINCT userId) numUsers,
    COUNT(DISTINCT movieId) numMovies,
    COUNT(*) totalRatings
FROM movielens.ratings

,numUsers,numMovies,totalRatings
0,138493,26744,20000263


### Exploring movie data

On examining the first few movies using the query following query, we can see that the genres column is a formatted string:

In [22]:
%%bigquery --project $PROJECT
SELECT *
FROM movielens.movies_raw
WHERE movieId < 5

,movieId,title,genres
0,3,Grumpier Old Men (1995),Comedy|Romance
1,4,Waiting to Exhale (1995),Comedy|Drama|Romance
2,2,Jumanji (1995),Adventure|Children|Fantasy
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


We can parse the genres into an array using the `SPLIT` function and create a new `movies` table as follows:

In [24]:
%%bigquery --project $PROJECT
CREATE OR REPLACE TABLE movielens.movies AS
    SELECT * REPLACE(SPLIT(genres, "|") AS genres)
    FROM movielens.movies_raw;
    
SELECT *
FROM movielens.movies
WHERE movieId < 5;

,movieId,title,genres
0,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]"
1,3,Grumpier Old Men (1995),"[Comedy, Romance]"
2,2,Jumanji (1995),"[Adventure, Children, Fantasy]"
3,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]"


BigQuery natively supports array data types and we're going to need them for our vectors next as part of matrix factorization.

## Matrix factorization
Matrix factorization is a collaborative filtering technique that relies on factorizing the ratings matrix into two vectors called the `user factors` and the `item factors`. The user factors is a low-dimensional representation of a user_id and the item factors similarly represents an item_id.

We can create the recommender model using [BigQuery ML](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create) and specifying `matrix_factorization` for the model type. Note that at the time of writing this model is still in Alpha and may not be available everywhere yet.

Run the below SQL code to create the recommendation model on movie data

__Note:__ Training the model __takes 30 minutes__. Feel free to skip this step as we have an already trained model you can use!

In [ ]:
%%bigquery --project $PROJECT
CREATE OR REPLACE MODEL movielens.recommender
options(model_type='matrix_factorization',
        user_col='userId', item_col='movieId', rating_col='rating')
AS

SELECT 
userId, movieId, rating
FROM movielens.ratings

In [25]:
%%bigquery --project $PROJECT
SELECT *
-- Note: remove cloud-training-demos if you are using your own model: 
FROM ML.TRAINING_INFO(MODEL `cloud-training-demos.movielens.recommender`)

,training_run,iteration,loss,eval_loss,duration_ms
0,0,4,0.531457,16.101844,211213
1,0,3,0.540680,19.061989,877333
2,0,2,0.555206,29.624823,214670
3,0,1,0.575037,167.012542,987004
4,0,0,66.468334,653898.138258,821665


Note that we create a model as usual, except that the model_type is matrix_factorization and that we have to identify which columns play what roles in the collaborative filtering setup.

What did you get? Our model took an hour to train, and the training loss starts out extremely bad and gets driven down to near-zero over next the four iterations:

<table>
  <tr>
    <th>Iteration</th>
    <th>Training Data Loss</th>
    <th>Evaluation Data Loss</th>
    <th>Duration (seconds)</th>
  </tr>
  <tr>
    <td>4</td>
    <td>0.5734</td>
    <td>172.4057</td>
    <td>180.99</td>
  </tr>
  <tr>
    <td>3</td>
    <td>0.5826</td>
    <td>187.2103</td>
    <td>1,040.06</td>
  </tr>
  <tr>
    <td>2</td>
    <td>0.6531</td>
    <td>4,758.2944</td>
    <td>219.46</td>
  </tr>
  <tr>
    <td>1</td>
    <td>1.9776</td>
    <td>6,297.2573</td>
    <td>1,093.76</td>
  </tr>
  <tr>
    <td>0</td>
    <td>63,287,833,220.5795</td>
    <td>168,995,333.0464</td>
    <td>1,091.21</td>
  </tr>
</table>

### Prevent the model from memorizing training data with regularization

However, the evaluation data loss is quite high, and much higher than the training data loss. This indicates that overfitting is happening, and so we need to add some [regularization](https://en.wikipedia.org/wiki/Regularization_(mathematics)). Let’s do that next. Note the added [l2_reg=0.2](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create#l2_reg)

(<b>Optional</b>, takes 30 minutes -- again you can use our trained model so you don't have to wait):

In [ ]:
%%bigquery --project $PROJECT
CREATE OR REPLACE MODEL movielens.recommender_l2
options(model_type='matrix_factorization',
        user_col='userId', item_col='movieId', 
        rating_col='rating', l2_reg=0.2)
AS

SELECT 
userId, movieId, rating
FROM movielens.ratings

In [27]:
%%bigquery --project $PROJECT
SELECT *
-- Note: remove cloud-training-demos if you are using your own model: 
FROM ML.TRAINING_INFO(MODEL `cloud-training-demos.movielens.recommender_l2`)

,training_run,iteration,loss,eval_loss,duration_ms
0,0,1,0.634794,0.749306,885965
1,0,0,66.530996,48894.170685,931206


Now, we get faster convergence (three iterations instead of five), and a lot less overfitting. Here are our results:

<table>
  <tr>
    <th>Iteration</th>
    <th>Training Data Loss</th>
    <th>Evaluation Data Loss</th>
    <th>Duration (seconds)</th>
  </tr>
  <tr>
    <td>2</td>
    <td>0.6509</td>
    <td>1.4596</td>
    <td>198.17</td>
  </tr>
  <tr>
    <td>1</td>
    <td>1.9829</td>
    <td>33,814.3017</td>
    <td>1,066.06</td>
  </tr>
  <tr>
    <td>0</td>
    <td>481,434,346,060.7928</td>
    <td>2,156,993,687.7928</td>
    <td>1,024.59</td>
  </tr>
</table>

### Increasing the number of factors for the model

By default, BigQuery sets the number of factors to be the log2 of the number of rows. In our case, since we have 20 million rows in the table, the number of factors would have been chosen to be 24. As with the number of clusters in K-Means clustering, this is a reasonable default but it is often worth experimenting with a number about 50% higher (36) and a number that is about a third lower (16):

In [ ]:
%%bigquery --project $PROJECT
CREATE OR REPLACE MODEL movielens.recommender_16
options(model_type='matrix_factorization',
        user_col='userId', item_col='movieId', 
        rating_col='rating', l2_reg=0.2, num_factors=16)
AS

SELECT 
userId, movieId, rating
FROM movielens.ratings

In [ ]:
%%bigquery --project $PROJECT
SELECT *
-- Note: remove cloud-training-demos if you are using your own model:
FROM ML.TRAINING_INFO(MODEL `cloud-training-demos.movielens.recommender_16`)

When we did that, we discovered that the evaluation loss was lower (0.97) with num_factors=16 than with num_factors=36 (1.67) or num_factors=24 (1.45). We could continue experimenting, but we are likely to see diminishing returns with further experimentation. So, let’s pick this as the final matrix factorization model and move on.

## Making recommendations

With the trained model, we can now provide recommendations. For example, let’s find the best comedy movies to recommend to the user whose userId is 903. In the query below, we are calling ML.PREDICT passing in the trained recommendation model and providing a set of movieId and userId to carry out the predictions on. In this case, it’s just one userId (903), but all movies whose genre includes Comedy.

In [ ]:
%%bigquery --project $PROJECT
SELECT * FROM
ML.PREDICT(MODEL `cloud-training-demos.movielens.recommender_16`, (
  SELECT 
    movieId, title, 903 AS userId
  FROM movielens.movies, UNNEST(genres) g
  WHERE g = 'Comedy'
))
ORDER BY predicted_rating DESC
LIMIT 5

## Filtering out already rated movies
Of course, this includes movies the user has already seen and rated in the past. Let’s remove them:


In [ ]:
%%bigquery --project $PROJECT
SELECT * FROM
ML.PREDICT(MODEL `cloud-training-demos.movielens.recommender_16`, (
  WITH seen AS (
    SELECT ARRAY_AGG(movieId) AS movies 
    FROM movielens.ratings
    WHERE userId = 903
  )
  SELECT 
    movieId, title, 903 AS userId
  FROM movielens.movies, UNNEST(genres) g, seen
  WHERE g = 'Comedy' AND movieId NOT IN UNNEST(seen.movies)
))
ORDER BY predicted_rating DESC
LIMIT 5

For this user, this happens to yield the same set of movies -- the top predicted ratings didn’t include any of the movies the user has already seen.

## Customer targeting

In the previous section, we looked at how to identify the top-rated movies for a specific user. Sometimes, we have a product and have to find the customers who are likely to appreciate it. Suppose, for example, we wish to get more reviews for movieId=96481 which has only one rating and we wish to send coupons to the 5 users who are likely to rate it the highest. We can identify those users using:

In [ ]:
%%bigquery --project $PROJECT
SELECT * FROM
ML.PREDICT(MODEL `cloud-training-demos.movielens.recommender_16`, (
  WITH allUsers AS (
     SELECT DISTINCT userId
     FROM movielens.ratings
  )
  SELECT 
    96481 AS movieId, 
    (SELECT title FROM movielens.movies WHERE movieId=96481) title,
    userId
  FROM
    allUsers
))
ORDER BY predicted_rating DESC
LIMIT 5

### Batch predictions for all users and movies
What if we wish to carry out predictions for every user and movie combination? Instead of having to pull distinct users and movies as in the previous query, a convenience function is provided to carry out batch predictions for all movieId and userId encountered during training. A limit is applied here, otherwise, all user-movie predictions will be returned and will crash the notebook.

In [ ]:
%%bigquery --project $PROJECT
SELECT *
FROM ML.RECOMMEND(MODEL `cloud-training-demos.movielens.recommender_16`)
LIMIT 10

As seen in a section above, it is possible to filter out movies the user has already seen and rated in the past. The reason already seen movies aren’t filtered out by default is that there are situations (think of restaurant recommendations, for example) where it is perfectly expected that we would need to recommend restaurants the user has liked in the past.

Copyright 2019 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.